## 과제 2: SageMaker Debugger 사용

이 실습에서는 Amazon SageMaker Debugger를 사용하여 훈련 작업 중에 발생하는 병목 현상, 리소스 사용률, 그리고 다양한 훈련 문제를 분석 및 탐지하고 관련 알림을 수신합니다.

### 과제 2.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
%%capture

#install updates
!apt-get update && apt-get install -y build-essential

**참고:** 패키지를 설치하는 데 5분 정도 걸릴 수 있습니다.

In [ ]:
%%capture

#install-dependencies

%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install -U sagemaker
%pip install -U smdebug
%pip install -U shap
%pip install protobuf==3.19.4

In [ ]:
#import libraries and set variable values

import sys
import boto3
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sagemaker
import sagemaker_datawrangler 
import shap

from mpl_toolkits.axes_grid1 import host_subplot

from sagemaker.debugger import (
    CollectionConfig,
    DebuggerHookConfig,
    FrameworkProfile,
    ProfilerConfig,
    ProfilerRule,
    Rule,
    rule_configs,
    TensorBoardOutputConfig
)

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sagemaker.xgboost.estimator import XGBoost
from smdebug.core import modes

base_job_name = "lab-7-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "lab-7-smdebugger"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
save_interval = 5

그런 다음, 데이터 집합을 가져옵니다.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

이 데이터 집합을 훈련(70%), 검증(20%), 검정(10%) 데이터 집합으로 분할합니다. 훈련 및 검증 데이터 집합은 이 실습에서 모델을 생성하는 데 사용됩니다. 검정 데이터 집합은 이 실습에서 사용하지 않습니다.

In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

feature_names = list(train_data.columns)

이제 Amazon Simple Storage Service(Amazon S3)에 해당 데이터 집합을 업로드합니다.

In [ ]:
#upload-dataset
sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

### 과제 2.2: SageMaker Debugger를 사용하도록 훈련 스크립트 수정

지정한 출력 S3 버킷에 텐서를 저장하도록 이전 실습에서 사용한 훈련 스크립트를 수정하고 저장할 텐서를 지정하며 디버그 후크를 등록해야 합니다.

모델을 훈련하려면 다음 항목을 구성해야 합니다.
- **Debugger Hook Parameters**: 훈련 단계에서 출력 텐서의 저장 간격을 조정하는 데 사용됩니다.
- **Debugger Rule Object**: 평가용 출력 텐서를 저장하는 데 사용됩니다.

**Debugger Hook Parameters**의 경우 훈련 중에 **metrics**, **feature_importance**, **full_shap** 및 **average_shap** 기본 제공 텐서 컬렉션을 수집하도록 구성합니다. **debugger_hook_config**의 **collection_configs**에서 이러한 항목을 구성합니다. **full_shap** 및 **average_shap** 기본 제공 텐서 컬렉션은 Shapley 값(SHAP)을 사용합니다. 기계 학습 예측에서 사용되는 SHAP는 훈련 데이터 인스턴스의 각 특성 값이 예측 결과를 생성하는 작업에 사용된다고 가정합니다. 그런 다음, 모든 특성에 결과를 균등하게 분산하는 방법을 지정합니다. SHAP에 관한 자세한 내용은 [설명 가능성을 위한 SHAP 기준](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)을 참조하세요.

**Debugger Rule Object**의 경우에는 **rules**에서 다음 **rule_configs**를 구성합니다.
- [Profiler Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report.html#debugger-profiling-report): 시스템 병목 현상 탐지를 위한 규칙을 실행하고 프로파일링 보고서를 자동 생성합니다.
- [XGBoost Training Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report-xgboost.html): 포괄적인 XGBoost 보고서를 실행합니다.
- [Overfit](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overfit): 검증 및 훈련 손실을 비교하여 모델이 훈련 데이터에 과대적합된 상태인지를 탐지합니다.
- [Overtraining](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overtraining): 모델이 과잉 훈련되고 있는지를 탐지합니다. 적절하게 작동하는 모델에서 훈련을 몇 번 반복하여 훈련 및 검증 손실이 모두 감소하면 해당 모델은 손실 최소값에 도달하여 더 이상 개선되지 않습니다. 이러한 모델을 계속 훈련하면 모델의 과대적합 현상이 시작되어 검증 손실이 증가하기 시작할 수 있습니다.
- [LossNotDecreasing](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#loss-not-decreasing): 손실 값이 적절한 비율로 감소하지 않는 경우를 탐지합니다. 이러한 손실은 스칼라 값이어야 합니다. 

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type='ml.m5.4xlarge',
    #Set the hyperparameters
    hyperparameters= {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "binary:logistic",
        "num_round": "300",
    },
    sagemaker_session=sagemaker_session,
    max_run=1800,
    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[  # For each of these, a new processing job will be run later in the lab
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
    ),
    #Set the Debugger Profiler Configuration
    profiler_config = ProfilerConfig(
        system_monitor_interval_millis=500,
        framework_profile_params=FrameworkProfile()

    ),
    #Configure the Debugger Rule Object
    rules = [
        ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
        Rule.sagemaker(rule_configs.create_xgboost_report()),  
        Rule.sagemaker(rule_configs.overfit()),
        Rule.sagemaker(rule_configs.overtraining()),
        Rule.sagemaker(rule_configs.loss_not_decreasing(),
            rule_parameters={
                "collection_names": "metrics",
                "num_steps": str(save_interval * 2),
            }
        )
    ]
)

### 과제 2.3: Debugger를 사용하도록 설정하고 훈련 작업 실행

이제 Debugger가 사용하도록 설정된 스크립트를 사용하여 XGBoost 모델을 훈련합니다. 훈련을 실행하려면 약 5-10분이 소요됩니다. 훈련 작업이 실행되는 동안 다음 과제를 계속 진행하면서 SageMaker Debugger를 사용하여 작업 진행률을 모니터링할 수 있습니다.

In [ ]:
#train-model
xgb.fit(
    inputs = data_inputs
)

### 과제 2.4: 훈련 작업 상태 모니터링

SageMaker Studio에서는 직접 시작한 모든 SageMaker Debugger 작업을 비롯한 시험 구성 요소를 검토할 수 있습니다. 이 실습에서는 **XGBoost Report** 작업, **Overfit** 작업, **Overtraining** 작업 및 **Loss Not Decreasing** 작업을 생성했습니다. SageMaker Studio에서 이러한 작업을 살펴봅니다.

다음 단계를 수행하면 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_7_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **lab_7_ko_kr.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 특성 그룹을 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다.

1. **SageMaker Home** 아이콘을 선택합니다.
2. **Experiments**를 선택합니다.

SageMaker Studio에서 **Experiments** 탭이 열립니다.

3. **Unassigned runs**를 선택합니다.
4. 목록에서 **Type**이 **SageMakerTrainingJob**인 작업의 **Name**을 선택합니다.

훈련 작업의 세부 정보가 표시됩니다.

5. 왼쪽에서 **Debugger** 탭을 선택합니다.

SageMaker Debugger는 훈련 작업 상태를 제공합니다. 모델 훈련이 실행되는 동안 이 상태를 모니터링할 수 있습니다. 훈련이 완료되면 지정한 훈련 문제의 상태가 표시됩니다.

모든 **Status** 줄에 **No Issues Found** 또는 **Issues Found**가 표시되면 분석이 완료된 것입니다. 디버거 규칙을 완료하는 데 9분 정도 걸릴 수 있습니다.

문제가 발견되면 모델에 수정해야 할 문제가 있다는 뜻입니다. 작업에 대해 발견된 문제가 있습니까? 

이 실습에서는 발견된 문제를 해결하지 않습니다. 하지만 발견된 문제를 해결하려는 경우 데이터 집합을 처리하고 조정한 하이퍼파라미터로 모델을 재훈련하여 문제를 해결할 수 있습니다.

6. 분석이 완료되면 레이블이 **lab_7_ko_kr.ipynb**인 노트북 탭으로 돌아옵니다.

### 과제 2.5: 훈련 후 분석 준비

SageMaker Debugger를 사용하면 Amazon CloudWatch에서 처리 작업 로그를 생성하여 사용자 지정 경보를 구성하는 데 사용할 수 있습니다. 여기서는 각 평가 대상 지표에 대해 로그가 저장되는 위치를 인쇄합니다.

In [ ]:
#print-urls
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
    """Helper function to get the rule job name with correct casing"""
    return "{}-{}-{}".format(
        training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
    )


def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(
        region, region, rule_job_name
    )


def get_rule_jobs_cw_urls(xgb):
    region = boto3.Session().region_name
    training_job = xgb.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]

    result = {}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(
                training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"]
            )
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(
                rule_job_name, region
            )
    return result


get_rule_jobs_cw_urls(xgb)

### 과제 2.6: SageMaker Debugger 인사이트 대시보드 액세스

1. **Experiments** 탭으로 돌아갑니다.
2. 왼쪽에서 **Debugger**를 선택합니다.
3. **Debugger insights** 섹션의 **Training job name** 목록에서 사용 가능한 **lab-7-smdebugger-job**을 선택합니다.

SageMaker Studio는 작업에 대한 새 **Debugger insights tab**을 열고 데이터 로드를 시작합니다.

SageMaker Debugger는 Amazon Elastic Compute Cloud(Amazon EC2) 인스턴스의 모델 훈련 성능 개요를 제공합니다. SageMaker Studio에서 SageMaker Debugger를 탐색하고 보고서에 포함된 세부 정보를 검사합니다.

**Systems Metrics** 탭에는 다음 섹션이 포함되어 있습니다.

- **Resource utilization summary**
- **CPU utilization summary**
- **GPU Utilization summary**

**Rules** 탭에 포함된 **Insights**는 다음과 같습니다. 

- **BatchSize**
- **LowGPUUtiliztion**
- **CPUBottleneck**
- **GPUMemoryIncrease**
- **StepOutlier**
- **MaxInitializationTime**
- **IOBottleneck**
- **LoadBalancing**

문제가 발견되면 차트와 테이블에 데이터가 입력됩니다.

4. **Debugger insights** 탭 상단 부근에 있는 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Download report</span>를 선택하여 Debugger 보고서를 다운로드할 수 있습니다.

### 마무리

축하합니다! SageMaker Debugger를 사용하여 모델 훈련에서 발생할 수 있는 문제를 분석 및 탐지하고 알림을 생성했습니다. 이제 보고서와 알림에서 생성된 정보를 사용해 모델을 효율적으로 훈련하여 더 효과적인 모델을 생성하는 데 필요한 인사이트를 제공할 수 있습니다. 다음 실습에서는 SageMaker Clarify를 사용하여 바이어스를 탐지하고 모델 예측을 위한 설명 가능성을 제공하는 방법을 중점적으로 살펴봅니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.